In [1]:
import wandb
import os

# wandb.init(project='VQA baseline (train)')
wandb.run.name = 'First wandb'
wandb.run.save()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hyeon_kki. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [3]:
import os
import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torchvision.models as models # 이미지
from torchvision import transforms # 아마지의 크기, 증강, 정규화를 진행하고 토치텐서로 출력하기 위해 사용
from PIL import Image

from transformers import GPT2Tokenizer, GPT2Model # 텍스트

from tqdm.auto import tqdm
import warnings 
warnings.filterwarnings('ignore')

from Modules.train import train
from Modules.inference import inference
from Modules.VQADataset import VQADataset
from Modules.VQAModel import VQAModel
from Modules.load_data import load_data

# init config setting
CFG = {
  "learning_rate": 5e-5,
  "epochs": 3,
  "batch_size": 64
}
wandb.config.update(CFG)

# device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"current device is {device}")

train_df, test_df, sample_submission, train_img_path, test_img_path = load_data()

# dataset & dataloaders
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# len(tokenizer)는 토크나이저가 인식하는 전체 토큰의 개수를 나타낸다.
# 즉, 토크나이저 어휘집의 크기와 동일합니다. 이 숫자는 모델이 학습된 토크나이저에 따라 달라진다.
# GPT2 토크나이저 어휘집 50258
vocab_size = len(tokenizer)

transform = transforms.Compose([ # 이미지 변환 함수들을 조합하여 하나의 변환 함수로 만들어주는 함수이다.
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])

train_dataset = VQADataset(train_df, tokenizer, transform, train_img_path, is_test=False)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Model
model = VQAModel(vocab_size).to(device)

# Criterion and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)



# Training loop
for epoch in range(3):
    avg_loss = train(model, train_loader, optimizer, criterion)
    wandb.log({"train_loss": avg_loss}, step=epoch)
    print(f"Epoch: {epoch+1}, Loss: {avg_loss:.4f}") # fprint 처리 잘하기 

/tmp/ipykernel_59076/2246612629.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/root/anaconda3/envs/QA/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


current device is cuda
current device is cuda
current device is cuda


100%|██████████| 5618/5618 [32:48<00:00,  2.85it/s]


Epoch: 1, Loss: 0.2197


100%|██████████| 5618/5618 [32:52<00:00,  2.85it/s]


Epoch: 2, Loss: 0.1687


100%|██████████| 5618/5618 [32:53<00:00,  2.85it/s]

Epoch: 3, Loss: 0.1552


In [4]:
# Dataset & DataLoader
test_dataset = VQADataset(test_df, tokenizer, transform, test_img_path, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# inference
preds = inference(model, test_loader)

no_pad_output = []
for pred in preds:
    output = pred[pred != 50257] # [PAD] token 제외
    no_pad_output.append(tokenizer.decode(output).strip()) # 토큰 id -> 토큰

100%|██████████| 633/633 [02:57<00:00,  3.57it/s]


In [ ]:
sample_submission['answer'] = no_pad_output
sample_submission.to_csv('submission.csv', index=False)

In [ ]:
solution = pd.read_csv('solution.csv')